In [ ]:
rephraser_prompt = """Your task is to rephrase the last user message from the conversation history  while removing the plan name and identify if user is asking question while ensuring the following:
1. Identify if user is asking question in last messages:
    - If last messages is question or conversation 

2. Remove Plan Names
    - If any specific plan name is mentioned in last or conversation history, do not include it in the rephrased version but you can include drug , provider or doctor name for search

3. Modify Multi-Plan Comparisons
    - If the user is comparing multiple plans , you need rephrase with single plan with below examples:
        User's original Question
        - "Which plan has lowest copay?" 
        - "Which plan has the lowest out of pocket maximum?" 
        - "List of copay of each plan" 
        - "Which plan has the lowest premium?",
        - "Which plan covers Dr. ABC?" 
        - "List the plans that cover Dr. XYZ"
        - "is doctor xyz covered?"


        Rephrased Questions
        - "What is the cost of this plan?"
        - "What is the out of pocket maximum?"
        - "What is the copay for this plan?"
        - "What is the premium for this plan?"
        - "Does this plan cover Dr. ABC"
        - "Does this plan cover Dr. XYZ?"
        -"is doctor xyz covered?"
    
5. Handle Ambiguities and Edge Cases
    - If the user's intent is unclear(e.g., indirect references to plans), infer the most relevant rephrased version while maintaining neutrality.
    - Do Not try to answer any question even you know the answer, Your task is to only rephrase 
    _ Make sure to think for classification type before responding

Output Format:
    - rephrased_message: provide the reworded version of the last messages.
    - is_question: "Whether or not the user is in the process of asking a question (last message need not be a question)"
    - question_type: clearly state listing_question or summary_question
"""
rephraser_prompt = """You are smart rephrasing assistant. Your task is to rewrite user's latest query as clear , standalone questions or statement. rephrase the last user message from the conversation history while removing any specific plan names and identifying if the user is asking a question. Ensure accuracy and clarity in the following steps:  
   
1. **Question Identification and Context Handling**:  
   - Analyze the entire conversation history to identify if the last message is a follow-up question or not.
   - If the latest messages depends on previous messages or full messages, (eg, it's follow-up or uses this , these,  that) then include necessary context in the rephrased query to make it fully understandable by it's own.
   - If it's not a follow-up question, simply rephrase the last message for clarity.  
   
2. **Remove Plan Names**:  
   - Omit any specific plan names mentioned in the last message or throughout the conversation history. You may include names of drugs, providers, or doctors to aid in clarity for potential searches.  
   
3. **Modify Multi-Plan Comparisons**:  
   - When a user is comparing multiple plans, rephrase to focus on a generic, single plan. Provide detailed examples to guide rephrasing:  
     - Original: "Which plan offers the best benefits for prescription drugs?"  
     - Rephrased: "What are the prescription drug benefits for this plan?"  
     - Original: "Can you compare the dental coverage in these plans?"  
     - Rephrased: "What is the dental coverage like in this plan?"  
     - Original: "Which plan should I choose for the lowest hospitalization costs?"  
     - Rephrased: "What are the hospitalization costs in this plan?"  

    - Follow up questions:
        - Original: Human: is crutches covered? AI: Yes Human: What is the cost each  plan?
        - Rephrased: "What is the cost of crutches in this plan?"
   
4. **Handle Ambiguities and Edge Cases**:  
   - If the user's intent is unclear or if there are indirect references to plans, infer the most relevant rephrased version while maintaining a neutral tone.  
   - Avoid attempting to answer any questions directly, focus solely on rephrasing for clarity and context.  
   
5. **Classification of Questions**:  
   - Accurately classify the type of question being asked as either a "listing_question" or a "summary_question" based on the nature of the inquiry.  
   
**Output Format**:  
- **rephrased_message**: Provide a reworded version of the last message that is clear, context-aware, and free of specific plan names.  
- **is_question**: Indicate "yes" if the user is in the process of asking a question or "no" if not. Note that the last message need not be a question itself.  
- **question_type**: Clearly state whether it is a "listing_question" or a "summary_question", based on the content and intent of the query.
"""

In [2]:
rephraser_prompt = """You are smart rephrasing assistant. Your task is to rewrite user's latest query as clear , standalone questions or statement. rephrase the last user message from the conversation history while removing any specific plan names and identifying if the user is asking a question. Ensure accuracy and clarity in the following steps:  
   
1. **Question Identification and Context Handling**:  
   - Analyze the entire conversation history to identify if the last message is a follow-up question or not.
   - If the latest messages depends on previous messages or full messages, (eg, it's follow-up or uses this , these,  that) then include necessary context in the rephrased query to make it fully understandable by it's own.
   - If it's not a follow-up question, simply rephrase the last message for clarity.  
   
2. **Remove Plan Names**:  
   - Omit any specific plan names mentioned in the last message or throughout the conversation history. You may include names of drugs, providers, or doctors to aid in clarity for potential searches.  
   
3. **Modify Multi-Plan Comparisons**:  
   - When a user is comparing multiple plans, rephrase to focus on a generic, single plan. Provide detailed examples to guide rephrasing:  
     - Original: "Which plan offers the best benefits for prescription drugs?"  
     - Rephrased: "What are the prescription drug benefits for this plan?"  
     - Original: "Can you compare the dental coverage in these plans?"  
     - Rephrased: "What is the dental coverage like in this plan?"  
     - Original: "Which plan should I choose for the lowest hospitalization costs?"  
     - Rephrased: "What are the hospitalization costs in this plan?"  

    - Follow up questions:
        - Original: **Human: is crutches covered? AI: Yes Human: What is the cost each  plan?**
        - Rephrased: **"What is the cost of crutches in this plan?"**

        - Original : **Human: Is crutches covered under this plan? AI: yes, crutches are covered under this plan Human: Is there a copay? AI: No Separate copay for crutches but it counts towards your deductible Human: How much is the deductible?**
        - Rephrased: **"What much is deductible  for crutches under this plan ?**
   
4. **Handle Ambiguities and Edge Cases**:  
   - If the user's intent is unclear or if there are indirect references to plans, infer the most relevant rephrased version while maintaining a neutral tone.  
   - Avoid attempting to answer any questions directly, focus solely on rephrasing for clarity and context.  
   
5. **Classification of Questions**:  
   - Accurately classify the type of question being asked as either a "listing_question" or a "summary_question" based on the nature of the inquiry.  
   
**Output Format**:  
- **rephrased_message**: Provide a reworded version of the last message that is clear, context-aware, and free of specific plan names.  
- **is_question**: Indicate "yes" if the user is in the process of asking a question or "no" if not. Note that the last message need not be a question itself.  
- **question_type**: Clearly state whether it is a "listing_question" or a "summary_question", based on the content and intent of the query.
"""
rephraser_classification_prompt = """ Classify the Question Type (Ensure Accurate Classification)	
	1. Listing Questions(Multiple Plans Compared or Listed)
    - Definition: If the user ask for details about multiple plans, it's A Listing Question
    - Trigger Keywords: "each", "all", "list", "list of which", "compare", "any of these plans", "among these plans", "for each of these plans"
    2. Summary Question(Single Plan Evaluation)
    - Definition: If the user ask which plan or what plan is the based based on a criterion, It is a Summary Question
    - Trigger Keywords: "which", "what"
    Below are Examples for Listing  ans Summary Question:
        - "List the copay for all plans" - Listing Question
        - "Do any of these plans cover dental" - Listing Question
        - "What's the highest copay among these plans"- Listing Question
        - "Compare the deductibles for each plans"- Listing Question
        - "Which plan has the lowest premium?"- Summary Question
        - "What plan offers the best coverage?"- Summary Question

    Priority Rules(Strictly Follow These):
    - if a Listing Keyword is present, it is ALWAYS a Listing Question, even if summary-related words exist.
    - If neither category fits, assume it is a Summary Question as the fallback
Output Format:
    - question_type: clearly state listing_question or summary_question
"""


from typing import Literal, Optional, List, Any, Dict

from langchain_openai import AzureChatOpenAI
from langchain_core.messages import SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from typing import Literal,Optional
import os
import json
import asyncio
# Workflow constants
NUM_FOLLOWUP_Q = 3
TOP_N_DOCUMENTS = 5

# LLM Stuff
API_KEY = os.getenv("OPENAI_KEY")
AZURE_ENDPOINT = os.getenv("OPENAI_ENDPOINT")
API_VERSION = "2025-01-01-preview"

rephraser_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(rephraser_prompt),
        MessagesPlaceholder("msgs"),
    ]
)




rephraser_classification_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(rephraser_classification_prompt),
        MessagesPlaceholder("msgs"),
    ]
)

class RephrasalQuestion(BaseModel):
    rephrased_message: str = Field(description="The user's rephrased complete message")
    is_question: bool = Field(
        description="Whether or not the user is in the process of asking a question (last message need not be a question)"
    )
    # question_type: Optional[str]


class Rephrasal(BaseModel):
    rephrased_message: str = Field(description="The user's rephrased complete message")
    is_question: bool = Field(
        description="Whether or not the user is in the process of asking a question (last message need not be a question)"
    )
    question_type: Optional[str]

class RephrasalClassification(BaseModel):
    question_type: Literal["listing_question", "summary_question"]


async def rephraser_classifier(msgs: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Rephrase the given messages and classify the type of question.

    Args:
        msgs (List[Dict[str, Any]]): A list of message dictionaries to be rephrased and classified.

    Returns:
        Dict[str, Any]: A dictionary containing the rephrased message, whether it is a question, and the question type.

    Raises:
        Exception: If there is an error during the rephrasing or classification process.
    """

    try:
        output_1, output_2 = await asyncio.gather(
            rephraser_chain.ainvoke(msgs),
          rephraser_chain_classification.ainvoke(msgs),
        )

        data = json.loads(output_1["raw"].content)
        data["question_type"] = list(json.loads(output_2["raw"].content).values())[0]
        output_1["raw"] = AIMessage(json.dumps(data))
        output_1["parsed"] = Rephrasal(
            rephrased_message=output_1["parsed"].model_dump()["rephrased_message"],
            is_question=output_1["parsed"].model_dump()["is_question"],
            question_type=output_2["parsed"].model_dump()["question_type"],
        )
        
        return output_1
    except Exception as e:
        # logging.error(f"Error in rephraser function: {e}")
        raise




llm = AzureChatOpenAI(
    api_key=API_KEY,  # type:ignore
    azure_endpoint=AZURE_ENDPOINT,
    azure_deployment="gpt-4o-mini",
    api_version=API_VERSION,
    temperature=0,
)

rephraser_chain:Any = rephraser_template | llm.with_structured_output(RephrasalQuestion, method="json_schema", include_raw=True)
rephraser_chain_classification:Any = rephraser_classification_template | llm.with_structured_output(RephrasalClassification, method="json_schema", include_raw=True)


In [3]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage

msgs = [HumanMessage("what is the copay for plan UHG 1234")]
output_1 = await rephraser_classifier(msgs)
output_1


{'raw': AIMessage(content='{"rephrased_message": "What is the copay for this plan?", "is_question": true, "question_type": "summary_question"}', additional_kwargs={}, response_metadata={}),
 'parsed': Rephrasal(rephrased_message='What is the copay for this plan?', is_question=True, question_type='summary_question'),
 'parsing_error': None}

In [4]:
msgs += [ HumanMessage("Which plan covers crutches?"),
    AIMessage("yes"),
       HumanMessage("What does it  cost for each plan?")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What is the cost of this plan?' is_question=True question_type='listing_question'


In [5]:
msgs = [HumanMessage("which plan has the lowest copay")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What is the copay amount for this plan?' is_question=True question_type='summary_question'


In [6]:
msgs = [HumanMessage("List te details of plan 1 , plan 2 and plan3 ")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])


rephrased_message='Please provide the details of this plan.' is_question=True question_type='listing_question'


In [7]:
msg = "How much will this cost me on a monthly basis?"

msgs = [HumanMessage(msg)]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])


rephrased_message='What is the monthly cost for this plan?' is_question=True question_type='summary_question'


In [8]:
msg = "Which plan has the lowest out of pocket maximum?"
msgs = [HumanMessage(msg)]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])


rephrased_message='What is the out-of-pocket maximum for this plan?' is_question=True question_type='summary_question'


In [9]:
msg = "list the copays for every plan in details"
msgs = [HumanMessage(msg)]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])


rephrased_message='What are the copays for this plan in detail?' is_question=True question_type='listing_question'


In [10]:
msg = "Can I see doctors out of the network?"
msgs = [HumanMessage(msg)]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])


rephrased_message='Am I allowed to see doctors who are not in the network?' is_question=True question_type='summary_question'


In [11]:
msg = "which plan has the lowest premium?"
msgs = [HumanMessage(msg)]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])


rephrased_message='What is the premium for this plan?' is_question=True question_type='summary_question'


In [12]:
msg = "What do I need to join this plan?"
msgs = [HumanMessage(msg)]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])


rephrased_message='What are the requirements to join this plan?' is_question=True question_type='summary_question'


In [13]:
msg = "How can I get paid back if I have to pay for a medical bill for  AARP Medicare Advantage from UHC KC-0005 (PPO)"
msgs = [HumanMessage(msg)]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])


rephrased_message='What is the process for getting reimbursed for a medical bill paid under a Medicare Advantage plan?' is_question=True question_type='summary_question'


In [14]:
msg = "List the details of AARP Medicare Advantage from UHC KC-0005 (PPO) , AARP Medicare Advantage Extras from UHC KC-5 (PPO) and AARP Medicare Advantage from UHC MN-0005 (PPO)"
msgs = [HumanMessage(msg)]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])


rephrased_message='What are the details of this Medicare Advantage plan?' is_question=True question_type='listing_question'


In [15]:
msg = "Are virtual visits covered under all of these plans?"
msgs = [HumanMessage(msg)]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])


rephrased_message='Are virtual visits covered under this plan?' is_question=True question_type='listing_question'


In [16]:
all_messages = [
    "what is the copay for plan UHG AARP Medicare Advantage from UHC MS-0003 (PPO)",
    "which plan has the lowest copay",
    "List the details of AARP Medicare Advantage from UHC KC-0005 (PPO) , AARP Medicare Advantage Extras from UHC KC-5 (PPO) and \
        AARP Medicare Advantage from UHC MN-0005 (PPO)",
    "How much will this cost me on a monthly basis?",
    "Which plan has the lowest out of pocket maximum?",
    "Can I see doctors out of the network ?",
    "which plan has the lowest premium?",
    "How can I get paid back if I have to pay for a medical bill for  AARP Medicare Advantage from UHC KC-0005 (PPO)",
    "What do I need to join this plan?",
    "Are virtual vists covered under in all three  plan?",
]

for msg in all_messages:
    msgs = [HumanMessage(msg)]
    output_1 = await rephraser_classifier(msgs) 
    print(output_1['parsed'])


rephrased_message='What is the copay for this Medicare Advantage plan?' is_question=True question_type='summary_question'
rephrased_message='What is the copay amount in this plan?' is_question=True question_type='summary_question'
rephrased_message='What are the details of this Medicare Advantage plan?' is_question=True question_type='listing_question'
rephrased_message='What is the monthly cost for this plan?' is_question=True question_type='summary_question'
rephrased_message='What is the out-of-pocket maximum for this plan?' is_question=True question_type='summary_question'
rephrased_message='Am I allowed to see doctors who are not in the network?' is_question=True question_type='summary_question'
rephrased_message='What is the premium for this plan?' is_question=True question_type='summary_question'
rephrased_message='What is the process for getting reimbursed for a medical bill paid under this Medicare Advantage plan?' is_question=True question_type='summary_question'
rephrased_me

In [17]:
msgs = [
    HumanMessage("Hi how are you"),
    AIMessage(
        "I'm doing well thank you. How can I help you look for Medicare Advantage plans today? You can ask about copays, coverage, costs, or anything else you might need."
    ),
    HumanMessage("Is eyewear covered under plan 0002?"),
    AIMessage(
        "Yes, this plan does offer coverage for eyewear. Under the plan, you are eligible for routine eyewear benefits provided through UnitedHealthcare Vision®. This includes either one pair of standard eyeglasses or one pair of contact lenses every two years with a $300 allowance toward the purchase. Standard frames and lenses are fully covered with a $0 copayment for the routine benefit, while upgrades such as deluxe frames or progressive lenses are not covered. Would you like more details about what qualifies as standard eyewear or any other aspects of the benefit?"
    ),
    HumanMessage(
        "Which plan has the lowest copay?"
    )
]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What is the copay for this plan?' is_question=True question_type='summary_question'


In [18]:
msgs += [
    AIMessage("The plan with the lowest copay for in network primary care vists is AARP Medicare Advantage from UHC CO-0001 (HMO-POS). Would you like more details about copay for various providers, such as specialists or mental health?"),
    HumanMessage("What about coinsurance?")
]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What is the coinsurance for this plan?' is_question=True question_type='summary_question'


In [19]:
msgs += [
    AIMessage("Under our plan, there are two different limits on what you have to pay out-of-pocket for covered medical services:\n\u00b7 Your in-network maximum out-of-pocket amount is $6,700. This is the most you pay during the calendar year for covered Medicare Part A and Part B services received from network providers. The amounts you pay for copayments and coinsurance for covered services from network providers count toward this in-network maximum out-of-pocket amount. The amounts you pay for Part D prescription drugs and services from out-of-network providers do not count toward your in-network maximum out-of-pocket amount. In addition, amounts you pay for some services do not count toward your in-network maximum out-of-pocket amount. These services are marked with an asterisk in the medical benefits chart. If you have paid $6,700 for covered Part A and Part B services from network providers, you will not have any out-of-pocket costs for the rest of the year when you see our network providers. However, you must continue to pay the Medicare Part B premium (unless your Part B premium is paid for you by Department of Health and Human Services (Medicaid) or another third party). \u00b7 Your combined maximum out-of-pocket amount is $6,700. This is the most you pay during the calendar year for covered Medicare Part A and Part B services received from both innetwork and out-of-network providers. The amounts you pay for copayments and coinsurance for covered services count toward this combined maximum out-of-pocket amount. The amounts you pay for your Part D prescription drugs do not count toward your combined maximum outof-pocket amount. In addition, amounts you pay for some services do not count toward your combined maximum out-of-pocket amount. These services are marked with an asterisk in the medical benefits chart. If you have paid $6,700 for covered services, you will have 100% coverage and will not have any out-of-pocket costs for the rest of the year for covered Part A and Part B services. However, you must continue to pay the Medicare Part B premium (unless your Part B premium is paid for you by Department of Health and Human Services (Medicaid) or another third party)."),
    HumanMessage("Am I covered for chiropractic services for  AARP Medicare Advantage from UHC KC-0008 (PPO)")
]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='Are chiropractic services covered under this plan?' is_question=True question_type='summary_question'


In [20]:
msgs += [
    AIMessage("Covered services include: \n\u00b7 Manual manipulation of the spine to correct subluxation (when one or more of the bones of your spine move out of position). Manual manipulation is a treatment that uses hands-on pressure to gently move your joints and tissues."),
    HumanMessage("Am I covered for colonoscopy for AARP Medicare Advantage from UHC MI-0001 (PPO)")
]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='Is colonoscopy covered under this plan?' is_question=True question_type='summary_question'


In [21]:
# 
msgs += [
     AIMessage("Let me know if I can help with something else"),
    HumanMessage("Do any of these plans cover crutches? ")
]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='Are crutches covered under this plan?' is_question=True question_type='listing_question'


In [22]:
msgs += [ AIMessage("What else"),
       HumanMessage("Which plan is cheaper on a monthly basis? ")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What is the monthly cost of this plan?' is_question=True question_type='summary_question'


In [23]:
msgs += [ AIMessage("What else"),
       HumanMessage("What are the key differences among these plans?")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What are the key differences in coverage and benefits among these plans?' is_question=True question_type='listing_question'


In [24]:
msgs += [ AIMessage("What else"),
       HumanMessage("Do any of these plans cover skilled nursing facilities? ")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='Is there coverage for skilled nursing facilities under this plan?' is_question=True question_type='listing_question'


In [25]:
msgs += [ AIMessage("What else"),
       HumanMessage("Are home health benefits covered under each of these plans? ")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='Are home health benefits included in this plan?' is_question=True question_type='listing_question'


In [26]:


msgs += [ AIMessage("What else"),
       HumanMessage("What is my physician copay? ")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What is the copay for my physician under this plan?' is_question=True question_type='summary_question'


In [27]:
msgs += [ AIMessage("What else"),
       HumanMessage("Summarize the key similarities and differences between these two plans.  ")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What are the key similarities and differences between these two plans?' is_question=True question_type='listing_question'


In [28]:
msgs += [ AIMessage("What else"),
       HumanMessage("List the out-of-pocket maximum for each plan")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What is the out-of-pocket maximum for this plan?' is_question=True question_type='listing_question'


In [29]:
msgs += [ AIMessage("What else"),
       HumanMessage("List all plan which has lowest cost for non-preferred generic drugs?   ")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What is the cost of non-preferred generic drugs in this plan?' is_question=True question_type='listing_question'


In [30]:


msgs += [ AIMessage("What else"),
       HumanMessage("Do any of these plans cover skilled nursing facilities? ")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='Is there coverage for skilled nursing facilities under this plan?' is_question=True question_type='listing_question'


In [31]:


msgs += [ AIMessage("What else"),
       HumanMessage("Are out of network providers covered under all of these plans? ")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='Are out-of-network providers covered under this plan?' is_question=True question_type='listing_question'


In [32]:


msgs += [ AIMessage("What else"),
       HumanMessage("Are home health benefits covered under each of these plans? ")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='Are home health benefits included in this plan?' is_question=True question_type='listing_question'


In [33]:


msgs += [ AIMessage("What else"),
       HumanMessage("are my drugs covered ?")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='Are my medications covered under this plan?' is_question=True question_type='summary_question'


# new test cases

In [34]:


msgs += [ AIMessage("What else"),
       HumanMessage("is doctor xyz covered?")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='Is this doctor covered under the plan?' is_question=True question_type='summary_question'


In [35]:
msgs += [ HumanMessage("Which plan covers crutches?"),
    AIMessage("yes"),
       HumanMessage("What does it  cost for each plan?")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What is the cost for this plan?' is_question=True question_type='listing_question'


In [36]:
msgs += [ HumanMessage("Does this plan cover insulin?"),
    AIMessage("yes, it's covered"),
       HumanMessage("And how much will it cost me ?")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What will be the cost of insulin under this plan?' is_question=True question_type='summary_question'


In [37]:
msgs += [ HumanMessage("Tell me about the urgent care visit?"),
    AIMessage("Urgent care visit is covered with 50$ copay"),
       HumanMessage("And Emergency Room?")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What is the coverage for emergency room visits under this plan?' is_question=True question_type='summary_question'


In [38]:
msgs += [ HumanMessage("Tell me about the urgent care visit?"),
    AIMessage("Urgent care visit is covered with 50$ copay"),
       HumanMessage("And ER??")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What is the coverage and cost for an emergency room visit under this plan?' is_question=True question_type='summary_question'


In [39]:
msgs += [ HumanMessage("Can I get dental for this plan"),
    AIMessage("No, it's not covered"),
       HumanMessage("What are the vision benefits?")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What are the vision benefits included in this plan?' is_question=True question_type='summary_question'


In [40]:
msgs += [ HumanMessage("Can I get dental for this plan"),
    AIMessage("No, it's not covered"),
       HumanMessage("I know")]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='I understand that dental is not covered under this plan.' is_question=False question_type='summary_question'


In [41]:
msgs = []
msgs += [ HumanMessage("Is crutches covered under this plan?"),
    AIMessage("yes, crutches are covered under this plan"),
       HumanMessage("Is there a copay?"),
       AIMessage("No Separate copay for crutches but it counts towards your deductible"),
       HumanMessage("How much is the deductible?"), 
    
       ]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What is the deductible amount for this plan?' is_question=True question_type='summary_question'


In [42]:
msgs = []
msgs += [ HumanMessage("Is crutches covered under this plan?"),
    AIMessage("yes, crutches are covered under this plan"),
       HumanMessage("Is there a copay?"),
       AIMessage("No Separate copay for crutches but it counts towards your deductible"),
       HumanMessage("How much is the deductible?"), 
        AIMessage("it's 1500$"),
         HumanMessage("How much should I pay after $1500 deductible?"), 
    
       ]
output_1 = await rephraser_classifier(msgs) 
print(output_1['parsed'])

rephrased_message='What will my out-of-pocket costs be after the $1500 deductible?' is_question=True question_type='summary_question'


In [43]:
msgs

[HumanMessage(content='Is crutches covered under this plan?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes, crutches are covered under this plan', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Is there a copay?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No Separate copay for crutches but it counts towards your deductible', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How much is the deductible?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="it's 1500$", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How much should I pay after $1500 deductible?', additional_kwargs={}, response_metadata={})]